In [ ]:
def main():
    # args = parse_args()

    # model_path = args.model_name_or_path
    model_path = "/opt/ml/outputs"
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForCausalLM.from_pretrained(model_path)
    # text = args.prompt_text
    text = "청계산 셰르파"
    input_ids = tokenizer.encode(text)
    # Check generation time
    start = time.time()
    gen_ids = model.generate(
        torch.tensor([input_ids]),
        max_length=64,
        repetition_penalty=2.0,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
        bos_token_id=tokenizer.bos_token_id,
        # num_beams=5,
        do_sample=True,
        top_k=30,
        top_p=0.95,
        use_cache=True,
    )
    generated_text = tokenizer.decode(gen_ids[0, :].tolist())
    end = time.time()

    print(generated_text)
    print(f"{end - start:.5f} sec")


if __name__ == "__main__":
    main()
